In [126]:
import pandas as pd
import nltk
from string import digits
from nltk.corpus import stopwords
import math
import numpy as np
import os
import sklearn
from sklearn.model_selection import KFold
import string
from sklearn.metrics import accuracy_score
from sklearn import metrics
from sklearn.metrics import classification_report
import random

In [127]:
def file_name(file_dir): 
    for root, dirs, files in os.walk(file_dir):
        print(root) #当前目录路径
        print(dirs) #当前路径下所有子目录
        print(files) #当前路径下所有非目录子文件
        return files

In [179]:
files = file_name('..\data')

..\data
[]
['AlarmClock.txt', 'Baby.txt', 'Bag.txt', 'CableModem.txt', 'Dumbbell.txt', 'F1.jpg', 'F1.png', 'Flashlight.txt', 'Gloves.txt', 'GPS.txt', 'GraphicsCard.txt', 'Headphone.txt', 'HomeTheaterSystem.txt', 'Jewelry.txt', 'Keyboard.txt', 'Magazine_Subscriptions.txt', 'Movies_TV.txt', 'Projector.txt', 'RiceCooker.txt', 'Sandal.txt', 'Vacuum.txt', 'Video_Games.txt']


In [130]:
def CleanLines(line):
    return line.lower() #换小写

In [131]:
def countFrequence(raw):
    review = raw['Review']
    label = raw['Label']
    
    words = CleanLines(review).split()
                
    for word in words:
        try:
            wordFreq.update({word:wordFreq.get(word)+1})
        except:
            wordFreq.update({word:1})
                
        if label == 'POS':
            try:
                posFreq.update({word:posFreq.get(word)+1})
            except:
                posFreq.update({word:1})
        elif label == 'NEG':
            try:
                negFreq.update({word:negFreq.get(word)+1})
            except:
                negFreq.update({word:1})                    

In [132]:
def calculateProba(review):
    words = CleanLines(review).split()
    
    posProba = math.log10(posPriorProb)
    negProba = math.log10(negPriorProb)
    for word in words:
        wordPosProba = posProb.get(word)
        if wordPosProba is None:
            posProba += math.log10( (lambdaSmoothing + 0)/(lambdaSmoothing*len(vocabulary) + posTotalFreq)) ##
        else:
            posProba += math.log10(wordPosProba)
        
        
        wordNegProba = negProb.get(word)
        if wordNegProba is None:
            negProba += math.log10((lambdaSmoothing + 0)/(lambdaSmoothing*len(vocabulary) + negTotalFreq))
        else:
            negProba += math.log10(wordNegProba)
    try:
        return (1-10**(negProba-posProba))/(1+10**(negProba-posProba)) ##normalize to -1,1
    except:
        if(negProba-posProba>20):
            return (1-10**int(negProba-posProba))/(1+10**int(negProba-posProba)) ##normalize to -1,1       

In [161]:
posCommon = {}
negCommon = {}
startID = 3
file = files[0]
trainData = pd.read_csv(file,sep='\t')

#initial learning
for file in files[1:startID]:
    trainData = pd.concat([trainData,pd.read_csv(file,sep='\t')])

    
ratio = 0.2

dataRow = trainData.shape[0]
print('******************************************************************')
print('ratio is: ',ratio)
print('Processing ',file,'with ',dataRow,' rows.')
X_train = trainData['Review']
y_train = trainData['Label']


#print('Shape of trainData: ',trainData.shape)
#print('Shape of testData: ',testData.shape)

wordFreq = {}
posFreq = {}
negFreq = {}
buffer =  trainData.apply(lambda X: countFrequence(X),axis =1)
#print('TrainData counting completed')

posVoca = list(posFreq.keys())
negVoca = list(negFreq.keys())
vocabulary = set(posVoca + negVoca)

posTotalFreq = 0
for word in posVoca:
    posTotalFreq += posFreq.get(word)

negTotalFreq = 0
for word in negVoca:
    negTotalFreq += negFreq.get(word)

lambdaSmoothing = 1

posProb = {}

for word in vocabulary:
    frequence = posFreq.get(word)
    if frequence is None:
        frequence = 0

    posProb.update({word: (lambdaSmoothing + frequence)/(lambdaSmoothing*len(vocabulary) + posTotalFreq)})

negProb = {}
wordProb = {}

for word in vocabulary:
    frequence = negFreq.get(word)
    if frequence is None:
        frequence = 0

    negProb.update({word: (lambdaSmoothing + frequence)/(lambdaSmoothing*len(vocabulary) + negTotalFreq)})

totalWords = sum(posFreq.values()) + sum(negFreq.values())

for word in vocabulary:
    try: 
        frequence = posFreq.get(word) + 0
        try:
            frequence += negFreq.get(word)
        except:
            pass
    except:
        frequence = negFreq.get(word)
    wordProb.update({word:frequence/totalWords})


## To calculate the orientation of words
orientation = {}
for word in vocabulary:
    if wordFreq.get(word) > 5*startID: #只出现一次的词偶然性太大
        orientation.update({word:posProb.get(word)/wordProb.get(word)})
    else:
        orientation.update({word:1})


#sorted() small to large, smaller means more negative 
threshold1 = (sorted(orientation.values()))[math.ceil(len(orientation)*ratio/2)]
threshold2 = (sorted(orientation.values()))[-math.ceil(len(orientation)*ratio/2)]
#print('threshold is: ', threshold)
#totalThreshold += threshold

#Only use the words whose orientation larger than threshold
for word in posProb:
    if (orientation.get(word) > threshold1) and  (orientation.get(word) < threshold2):
        posProb.update({word: (lambdaSmoothing + 0)/(lambdaSmoothing*len(vocabulary) + posTotalFreq)})

for word in negFreq:
    if (orientation.get(word) > threshold1) and  (orientation.get(word) < threshold2):
        negProb.update({word: (lambdaSmoothing + 0)/(lambdaSmoothing*len(vocabulary) + negTotalFreq)})

negValues = np.array((sorted(orientation.values())))[:math.ceil(len(orientation)*ratio/2)]
posValues  = np.array((sorted(orientation.values())))[-math.ceil(len(orientation)*ratio/2):]

for word in vocabulary:
    if (orientation.get(word) > threshold2):
        try:
            weight = 1 + np.where(posValues==orientation.get(word))[0][0]/len(posValues)
            posCommon.update({word:posCommon.get(word)+weight})
        except:
            posCommon.update({word:1})
    elif (orientation.get(word) < threshold1):
        try:
            weight = 1 + -(np.where(negValues==orientation.get(word))[0][0]-len(posValues))/len(posValues)
            negCommon.update({word:negCommon.get(word)+weight})
        except:
            negCommon.update({word:1})

posNum = sum(trainData['Label'] == 'POS')
negNum = sum(trainData['Label'] == 'NEG')
posPriorProb = posNum/(posNum+negNum)
negPriorProb = negNum/(posNum+negNum)

******************************************************************
ratio is:  0.2
Processing  Bag.txt with  3000  rows.


In [163]:
posCommon = {}
negCommon = {}

scores = np.zeros(21-startID)
k = 0
f1Score = 0

#initial learning
for file in files[startID:21]:
    trainData = pd.read_csv(file,sep='\t')

    ratio = 0.2

    dataRow = trainData.shape[0]
    print('******************************************************************')
    print('ratio is: ',ratio)
    print('Processing ',file,'with ',dataRow,' rows.')
    X_train = trainData['Review']
    
    y_train = [calculateProba(review) for review in X_train]
    y_train = ['POS' if tem>0.5 else 'NEG' if tem<-0.5 else 'NEU' for tem in y_train]

    #print('Shape of trainData: ',trainData.shape)
    #print('Shape of testData: ',testData.shape)

    buffer =  trainData.apply(lambda X: countFrequence(X),axis =1)
    #print('TrainData counting completed')

    posVoca = list(posFreq.keys())
    negVoca = list(negFreq.keys())
    vocabulary = set(posVoca + negVoca)

    posTotalFreq = 0
    for word in posVoca:
        posTotalFreq += posFreq.get(word)

    negTotalFreq = 0
    for word in negVoca:
        negTotalFreq += negFreq.get(word)

    lambdaSmoothing = 1

    posProb = {}

    for word in vocabulary:
        frequence = posFreq.get(word)
        if frequence is None:
            frequence = 0

        posProb.update({word: (lambdaSmoothing + frequence)/(lambdaSmoothing*len(vocabulary) + posTotalFreq)})

    negProb = {}
    wordProb = {}

    for word in vocabulary:
        frequence = negFreq.get(word)
        if frequence is None:
            frequence = 0

        negProb.update({word: (lambdaSmoothing + frequence)/(lambdaSmoothing*len(vocabulary) + negTotalFreq)})

    totalWords = sum(posFreq.values()) + sum(negFreq.values())

    for word in vocabulary:
        try: 
            frequence = posFreq.get(word) + 0
            try:
                frequence += negFreq.get(word)
            except:
                pass
        except:
            frequence = negFreq.get(word)
        wordProb.update({word:frequence/totalWords})


    ## To calculate the orientation of words
    orientation = {}
    for word in vocabulary:
        if wordFreq.get(word) > 5*(k+startID): #只出现一次的词偶然性太大
            orientation.update({word:posProb.get(word)/wordProb.get(word)})
        else:
            orientation.update({word:1})


    #sorted() small to large, smaller means more negative 
    threshold1 = (sorted(orientation.values()))[math.ceil(len(orientation)*ratio/2)]
    threshold2 = (sorted(orientation.values()))[-math.ceil(len(orientation)*ratio/2)]
    #print('threshold is: ', threshold)
    #totalThreshold += threshold

    #Only use the words whose orientation larger than threshold
    for word in posProb:
        if (orientation.get(word) > threshold1) and  (orientation.get(word) < threshold2):
            posProb.update({word: (lambdaSmoothing + 0)/(lambdaSmoothing*len(vocabulary) + posTotalFreq)})

    for word in negFreq:
        if (orientation.get(word) > threshold1) and  (orientation.get(word) < threshold2):
            negProb.update({word: (lambdaSmoothing + 0)/(lambdaSmoothing*len(vocabulary) + negTotalFreq)})

    negValues = np.array((sorted(orientation.values())))[:math.ceil(len(orientation)*ratio/2)]
    posValues  = np.array((sorted(orientation.values())))[-math.ceil(len(orientation)*ratio/2):]

    for word in vocabulary:
        if (orientation.get(word) > threshold2):
            try:
                weight = 1 + np.where(posValues==orientation.get(word))[0][0]/len(posValues)
                posCommon.update({word:posCommon.get(word)+weight})
            except:
                posCommon.update({word:1})
        elif (orientation.get(word) < threshold1):
            try:
                weight = 1 + -(np.where(negValues==orientation.get(word))[0][0]-len(posValues))/len(posValues)
                negCommon.update({word:negCommon.get(word)+weight})
            except:
                negCommon.update({word:1})

    posNum = sum(trainData['Label'] == 'POS')
    negNum = sum(trainData['Label'] == 'NEG')
    posPriorProb = posNum/(posNum+negNum)
    negPriorProb = negNum/(posNum+negNum)
    
    testData = pd.read_csv(file,sep='\t')
    testData = testData[testData['Label'] != 'NEU']
    print('******************************************************************')
    print('Processing ',file,'with ',dataRow,' rows.')
    predict = [calculateProba(review) for review in testData['Review']]
    result = np.zeros([len(predict),2])
    result = pd.DataFrame(result)
    result[0]=[1 if tem>0 else 0 for tem in predict]
    result[1]=list(testData['Label'])
    result[1] = [ 1 if (tem == 'POS') else 0 for tem in list(result[1])]
    target_names = ['class 0', 'class 1']
    print(classification_report(result[1], result[0], target_names=target_names))
    print(sklearn.metrics.f1_score(result[1],result[0], average='macro'))
    scores[k] = round(sklearn.metrics.f1_score(result[1],result[0], average='macro'),4)
    
    f1Score += sklearn.metrics.f1_score(result[1],result[0], average='macro')
    k += 1
scores[-1]=f1Score/k
print(scores)

******************************************************************
ratio is:  0.2
Processing  CableModem.txt with  1000  rows.
******************************************************************
Processing  CableModem.txt with  1000  rows.
              precision    recall  f1-score   support

     class 0       0.94      0.80      0.87       121
     class 1       0.97      0.99      0.98       845

   micro avg       0.97      0.97      0.97       966
   macro avg       0.96      0.90      0.92       966
weighted avg       0.97      0.97      0.97       966

0.924253512880562
******************************************************************
ratio is:  0.2
Processing  Dumbbell.txt with  1000  rows.
******************************************************************
Processing  Dumbbell.txt with  1000  rows.
              precision    recall  f1-score   support

     class 0       0.93      0.73      0.82       146
     class 1       0.95      0.99      0.97       764

   micro avg     

In [156]:
scores

array([0.8694    , 0.8748    , 0.8259    , 0.785     , 0.9121    ,
       0.8768    , 0.8858    , 0.9236    , 0.7599    , 0.8707    ,
       0.8932    , 0.8381    , 0.8575    , 0.8475    , 0.8059    ,
       0.8992    , 0.9068    , 0.86071391])

In [167]:
a=sorted(negCommon.items(), key = lambda x: x[1], reverse = True)[:20]

In [173]:
for i in a:
    print(i[0],' & ',i[1],'\\\ ')
 

refund  &  32.99921813917123 \\ 
garbage  &  32.994266353922335 \\ 
junk  &  32.985405264529575 \\ 
waste  &  32.984102163148286 \\ 
worst  &  32.97185301016418 \\ 
rma  &  32.96846494657285 \\ 
poorly  &  32.96194943966641 \\ 
terrible  &  32.95569455303623 \\ 
disappointed  &  32.949960906958566 \\ 
trash  &  32.948918425853535 \\ 
useless  &  32.94683346364347 \\ 
worthless  &  32.94057857701329 \\ 
awful  &  32.92520198071411 \\ 
defective  &  32.917904612978894 \\ 
return  &  32.913734688558776 \\ 
exchange  &  32.908001042481104 \\ 
respond  &  32.90487359916601 \\ 
poor  &  32.90409173833724 \\ 
disappointment  &  32.90278863695596 \\ 
crap  &  32.89653375032577 \\ 
